In [1]:
import intent
import State

c:\Users\jaime\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
WELCOME = "welcome"
SEARCH = "search"
LIST = "list"
RECIPE = "recipe"
INGREDIENTS = "ingredients"
DESC = "description"
STEPS = "steps"
STOP = "stop"

In [3]:
YES_EVENT = "YesIntent"
NO_EVENT = "NoIntent"
IDENTIFY_EVENT = "IdentifyProcessIntent"
WELCOME_EVENT = "GreetingIntent"
NOTFOUND_EVENT = "OutOfScopeIntent"
START_STEPS_EVENT = "StartStepsIntent"
NEXT_EVENT = "NextStepIntent"
PREVIOUS_EVENT = "PreviousStepIntent"
STOP_EVENT = "StopIntent"
RESTART = "RepeatIntent"
SELECT = "SelectIntent"

In [4]:
welcome_dict =  {(WELCOME, YES_EVENT): SEARCH}
welcome_intents = [YES_EVENT]
welcome_message = "Hello! Would you like to search for a recipe?"
Welcome_state = State.State(welcome_intents,welcome_message,WELCOME,welcome_dict)


In [5]:
search_dict = {(SEARCH, IDENTIFY_EVENT): LIST, (SEARCH, STOP_EVENT): STOP, (SEARCH, RESTART): WELCOME}
search_intents = [IDENTIFY_EVENT, STOP_EVENT, RESTART]
search_message = "That's great! What would you like to search for?"
Search_state = State.State(search_intents,search_message,SEARCH,search_dict)

In [6]:
list_dict = {(LIST, SELECT): RECIPE, (LIST, STOP_EVENT): STOP, (LIST, RESTART): WELCOME}
list_intents = [SELECT, STOP_EVENT, RESTART]
list_message = "Here is the list of recipes. Which one would you like to see?"
List_state = State.State(list_intents,list_message,LIST,list_dict)

In [7]:
recipe_dict = {(RECIPE, IDENTIFY_EVENT): LIST, (RECIPE, STOP_EVENT): STOP, (RECIPE, RESTART): WELCOME}
recipe_intents = [IDENTIFY_EVENT, STOP_EVENT, RESTART]
recipe_message = "Here is the recipe. Would you like to see the ingredients, steps, description or curiosity?"
Recipe_state = State.State(recipe_intents,recipe_message,RECIPE,recipe_dict)

In [8]:
steps_dict = {(SEARCH, IDENTIFY_EVENT): LIST, (SEARCH, NO_EVENT): STOP, (SEARCH, RESTART): WELCOME}
steps_intents = [IDENTIFY_EVENT, NO_EVENT, RESTART]
steps_message = "That's great! What would you like to search for?"
steps_state = State.State(search_intents,search_message,SEARCH,search_dict)

In [9]:
desc_dict = {(SEARCH, IDENTIFY_EVENT): LIST, (SEARCH, NO_EVENT): STOP, (SEARCH, RESTART): WELCOME}
desc_intents = [IDENTIFY_EVENT, NO_EVENT, RESTART]
desc_message = "That's great! What would you like to search for?"
desc_state = State.State(search_intents,search_message,SEARCH,search_dict)

In [10]:
ingredients_dict = {(SEARCH, IDENTIFY_EVENT): LIST, (SEARCH, NO_EVENT): STOP, (SEARCH, RESTART): WELCOME}
ingredients_intents = [IDENTIFY_EVENT, NO_EVENT, RESTART]
ingredients_message = "That's great! What would you like to search for?"
Ingredients_state = State.State(search_intents,search_message,SEARCH,search_dict)

In [11]:
States = {WELCOME: Welcome_state, SEARCH: Search_state, LIST: List_state, RECIPE: Recipe_state, INGREDIENTS: Ingredients_state, DESC: desc_state, STEPS: steps_state, STOP: None}

In [12]:
#intent.run("the third looks good")

In [13]:
import connect
client, index_name = connect.get_data()

class DialogManager():

    UNKNOWN_OPT = "Unkonwn option. Please try again."

    def __init__(self):
        self.state = States[WELCOME]
        self.current_Intent = None
        self.input = None
        self.intent = None
        self.recipe_list = None
        self.recipe = None
        self.first = True
    
    def get_option(self):
        if "1st" in self.input or "first" in self.input or "1" in self.input:
            self.recipe = self.recipe_list[0]
        elif "2nd" in self.input or "second" in self.input or "2" in self.input:
            self.recipe = self.recipe_list[1]
        elif "3rd" in self.input or "third" in self.input or "3" in self.input:
            self.recipe = self.recipe_list[2]
        elif "4th" in self.input or "fourth" in self.input or "4" in self.input:
            self.recipe = self.recipe_list[3]
        elif "5th" in self.input or "fifth" in self.input or "5" in self.input or "last" in self.input:
            self.recipe = self.recipe_list[4]
        else:
            self.recipe = self.UNKNOWN_OPT
    
    def run(self):
        while self.current_Intent != STOP_EVENT:
            print(self.state._message)
            self.input = str(input())   #input
            self.intent = intent.run(self.input)    #predict intent
            self.state.set_intent(self.intent)      #set intent
            self.current_Intent = self.state.get_intent()   #change current
            if self.current_Intent == STOP_EVENT or (self.state == Welcome_state and self.intent == NO_EVENT):
                break
            elif self.state.isStateIntent():    #state with something to do
                self.state = States[self.state.nextState()]

                if(self.first):
                    if self.state.get_state() == RECIPE:
                        self.get_option()
                        self.first = False
                        if self.recipe == self.UNKNOWN_OPT:
                            print(self.UNKNOWN_OPT)
                            continue

                self.recipe_list = self.state.execute(client, index_name, self.input, self.recipe)
            
                #print("Next state: " + self.state._state)
            else:
                print("Couldn't understand what you were looking for. Please try again.")
                continue
        print("Goodbye!") 

In [14]:
dm = DialogManager()

In [15]:
#I want a pizza recipe
#3rd

dm.run()

Hello! Would you like to search for a recipe?

That's great! What would you like to search for?
Couldn't understand what you were looking for. Please try again.
That's great! What would you like to search for?

'How To Make The Best Homemade Pizza'

'How To Make Homemade Pizza Sauce'

'Home-Made Pizza'

'Pizza Dough I'

'Dessert Pizza'

Here is the list of recipes. Which one would you like to see?

Ask something!

i want the instructions

"Ready to dive in? Step 1: \u201cBloom\u201d the yeast by sprinkling the sugar and yeast in the warm water. Let sit for 10 minutes, until bubbles form on the surface. "


next step

"Step 2: In a large bowl, combine the flour and salt. Make a well in the middle and add the olive oil and bloomed yeast mixture. Using a spoon, mix until a shaggy dough begins to form. "


last step

"The last step is here! Step 10: Sprinkle semolina onto an upside down baking sheet and place the stretched crust onto it. Add the sauce and ingredients of your choice. "


i 